In [1]:
import requests
import os
import pandas as pd

FSbrukar = os.environ["FSbrukar"]
FSpassord = os.environ["FSpassord"]

In [2]:
def query_FS_graphql(query, variable):
    hode = {
        'Accept': 'application/json;version=1',
        "Feature-Flags": "beta, experimental"
    }
    GraphQLurl = "https://api.fellesstudentsystem.no/graphql/"
    svar = requests.post(
        GraphQLurl, 
        json = {
            'query': query,
            'variables': variable
        },
        headers=hode,
        auth=(FSbrukar, FSpassord))
    if 200 <= svar.status_code < 300:
        return svar.json()
    else:
        return {}

In [3]:
query = """
query Emneansvarleg($after: String, $first: Int) {
  emner(
    filter: {eierInstitusjonsnummer: "203", ikkeUtloptITermin: {arstall: 2025, terminbetegnelse: HOST}}
    first: $first
    after: $after
  ) {
    pageInfo {
      hasNextPage
      endCursor
    }
    nodes {
      personroller(filter: {erAktiv: true}) {
        fsRolle {
          navn {
            publiseringsnavn
          }
          erAktiv
        }
        fagperson {
          navn {
            etternavn
            fornavn
          }
          feideBruker
        }
        emne {
          navnAlleSprak {
            nn
          }
          kode
        }
      }
    }
  }
}
"""

In [18]:
dataliste = []
n = 1
start = None
hentmeir = True
antal_per_side = 100
while hentmeir:
    variable = {'antal': antal_per_side, 'start': start}
    print(f"Henter emner {n} - {n + antal_per_side - 1}")
    svar = query_FS_graphql(query, variable)
    start = svar['data']['emner']['pageInfo']['endCursor']
    hentmeir = svar['data']['emner']['pageInfo']['hasNextPage']
    n += antal_per_side
    # Hente ut data
    data = svar['data']['emner']['nodes']
    for e in data:
        for p in e['personroller']:
            emnekode = p['emne']['kode']
            emnenavn = p['emne']['navnAlleSprak']['nn']
            ansvarleg = p['fagperson']['feideBruker']
            dataliste.append([emnekode, emnenavn, ansvarleg])

print(f"Henta ut {len(dataliste)} emner")


Henter emner 1 - 100
Henter emner 101 - 200
Henter emner 201 - 300
Henter emner 301 - 400
Henter emner 401 - 500
Henter emner 501 - 600
Henter emner 601 - 700
Henter emner 701 - 800
Henter emner 801 - 900
Henter emner 901 - 1000
Henter emner 1001 - 1100
Henter emner 1101 - 1200
Henter emner 1201 - 1300
Henter emner 1301 - 1400
Henter emner 1401 - 1500
Henter emner 1501 - 1600
Henter emner 1601 - 1700
Henter emner 1701 - 1800
Henter emner 1801 - 1900
Henter emner 1901 - 2000
Henter emner 2001 - 2100
Henter emner 2101 - 2200
Henter emner 2201 - 2300
Henter emner 2301 - 2400
Henter emner 2401 - 2500
Henter emner 2501 - 2600
Henter emner 2601 - 2700
Henter emner 2701 - 2800
Henter emner 2801 - 2900
Henter emner 2901 - 3000
Henter emner 3001 - 3100
Henter emner 3101 - 3200
Henter emner 3201 - 3300
Henter emner 3301 - 3400
Henter emner 3401 - 3500
Henter emner 3501 - 3600
Henter emner 3601 - 3700
Henter emner 3701 - 3800
Henter emner 3801 - 3900
Henter emner 3901 - 4000
Henter emner 4001 - 4

KeyboardInterrupt: 

In [21]:
dataliste[-1]

['BIO173', 'Organisk kjemi og biokjemi', 'estr@hvl.no']